# base data

download the data from net - test phase

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
from binance.client import Client
import pandas as pd
import time
import os
import urllib3
import json

# =============================================================================
# DISABLE SSL WARNINGS (SINCE verify=False IS USED)
# =============================================================================
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# =============================================================================
# LOAD API KEYS FROM EXTERNAL JSON FILE
# =============================================================================
keys_path = "C:/connection/binance_keys.json"

with open(keys_path, 'r') as f:
    keys = json.load(f)

API_KEY    = keys['api_key']
API_SECRET = keys['api_secret']

# =============================================================================
# INITIALIZE BINANCE CLIENT WITH DISABLED CERTIFICATE VERIFICATION
# =============================================================================
client = Client(API_KEY, API_SECRET, {"verify": False})

In [3]:
# =============================================================================
# DEFINE TIME RANGE FOR HISTORICAL DATA
# =============================================================================
from datetime import datetime

symbol   = 'BCHUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE
start    = '1 Jan, 2017'
end      = datetime.now().strftime("%d %b, %Y")

# =============================================================================
# FETCH FULL HISTORICAL 1-MINUTE CANDLE DATA FROM BINANCE
# =============================================================================
raw_data = client.get_historical_klines(
    symbol, interval, start, end
)


In [6]:
# =============================================================================
# CONVERT RAW DATA TO CLEANED DATAFRAME
# =============================================================================
import pandas as pd
import os

# Create DataFrame with appropriate column names
df = pd.DataFrame(
    raw_data,
    columns = [
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base', 'taker_buy_quote', 'ignore'
    ]
)

# Convert timestamps to datetime
df['open_time']  = pd.to_datetime(df['open_time'] , unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

# Convert numeric columns to float
numeric_cols = ['open', 'high', 'low', 'close', 'volume']
df[numeric_cols] = df[numeric_cols].astype(float)

# Keep only relevant columns
df = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]

# =============================================================================
# SAVE DATAFRAME TO PARQUET FILE IN 'data' FOLDER
# =============================================================================
os.makedirs('data', exist_ok=True)

output_path = 'data/bchusdt_1m_full.parquet'
df.to_parquet(output_path, index=False)

print(f"Data saved to {output_path}")


Data saved to data/bchusdt_1m_full.parquet
